In [1]:
import os
import json
import zipfile
import shutil
import glob
import pandas as pd
import numpy as np
import cv2
import yaml
from tqdm import tqdm

In [2]:
!pip install ultralytics

In [3]:
from ultralytics import YOLO, checks, hub
checks()

Ultralytics YOLOv8.0.151 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.5/166.8 GB disk)


In [4]:
SEED = 42
BATCH_SIZE = 4
MODEL = "trained_model" # 학습한 모델에 대한 정보
EPOCH = 300

In [6]:
!unzip '/content/drive/MyDrive/Colab Notebooks/21 ~ 24주차 - 3차 프로젝트/all_data_5.zip' -d all_5

Archive:  /content/drive/MyDrive/Colab Notebooks/21 ~ 24주차 - 3차 프로젝트/all_data_5.zip
   creating: all_5/test_images/
  inflating: all_5/test_images/064442001.png  
  inflating: all_5/test_images/065750973.png  
  inflating: all_5/test_images/071126840.png  
  inflating: all_5/test_images/071745784.png  
  inflating: all_5/test_images/072544230.png  
  inflating: all_5/test_images/073006756.png  
  inflating: all_5/test_images/073652331.png  
  inflating: all_5/test_images/074234995.png  
  inflating: all_5/test_images/074830954.png  
  inflating: all_5/test_images/075150865.png  
  inflating: all_5/test_images/075525438.png  
  inflating: all_5/test_images/075805469.png  
  inflating: all_5/test_images/080113199.png  
  inflating: all_5/test_images/080319671.png  
  inflating: all_5/test_images/080753161.png  
  inflating: all_5/test_images/081110250.png  
  inflating: all_5/test_images/081649493.png  
  inflating: all_5/test_images/081946762.png  
  inflating: all_5/test_images

In [23]:
# 작업한 파일 담을 폴더 생성
if os.path.exists("./data"):
    shutil.rmtree("./data")

if not os.path.exists("./data/train"):
    os.makedirs("./data/train")

if not os.path.exists("./data/val"):
    os.makedirs("./data/val")

if not os.path.exists("./data/test"):
    os.makedirs("./data/test")

if not os.path.exists("./results"):
    os.makedirs("./results")

In [5]:
# txt 파일을 욜로 학습용 txt로 변환
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape

        target_image_path = f"./data/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)

        if type == "test":
            continue

        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")

        target_label_txt = f"./data/{type}/{os.path.basename(txt_path)}"
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [7]:
# 원본 이미지와 텍스트가 있는 경로
train_images_paths = glob.glob("/content/all_5/train_images_5/*.png")
train_txt_paths = glob.glob("/content/all_5/train_txt_5/*.txt")

valid_images_paths = glob.glob("/content/all_5/val_images/*.png")
valid_txt_paths = glob.glob("/content/all_5/val_txt/*.txt")

make_yolo_dataset(train_images_paths, train_txt_paths, "train")
make_yolo_dataset(valid_images_paths, valid_txt_paths, "val")
make_yolo_dataset(sorted(glob.glob("/content/all_5/test_images/*.png")), None, "test")

100%|██████████| 340/340 [01:02<00:00,  5.41it/s]


In [8]:
import torch

# 저장한 모델 불러 오기
save_path = "/content/drive/MyDrive/Colab Notebooks/21 ~ 24주차 - 3차 프로젝트/yolo_model_0.pth"
loaded_model = torch.load(save_path)

In [9]:
with open("/content/classes.txt", "r") as reader: # 클래스 정보가 있는 경로
    lines = reader.readlines()
    classes = [line.strip().split(",")[1] for line in lines]

# yaml 데이터 생성
yaml_data = {
              "names": classes,
              "nc": len(classes),
              "train": "/content/data/train/", # 이미지와 욜로용 txt가 있는 폴더 경로 설정
              "val": "/content/data/val/",
              "test": "/content/data/test",
              "hsv_h": 0.015,  # image HSV-Hue augmentation (fraction)
              "hsv_s": 0.7,  # image HSV-Saturation augmentation (fraction)
              "hsv_v": 0.4,  # image HSV-Value augmentation (fraction)
              "degrees": 0.5,  # image rotation (+/- deg)
              "translate": 0.1,  # image translation (+/- fraction)
              "scale": 0.2,  # image scale (+/- gain)
              "fliplr": 0.5, # image flip left-right (probability)
              "mosaic": 0.3,  # image mosaic (probability)
              "mixup": 0.1  # image mixup (probability)
            }

with open("/content/data/data.yaml", "w") as writer: # 위에서 만든 yaml 데이터 경로 설정
    yaml.dump(yaml_data, writer)

In [14]:
# 욜로 모델 학습: 첫 번째
model = YOLO("yolov8n")
print(type(model.names), len(model.names))

print(model.names)
results = model.train(
    data="/content/data/data.yaml", # yaml 데이터가 있는 경로
    imgsz=640,
    epochs=EPOCH,
    batch=BATCH_SIZE,
    # patience=5,
    workers=16,
    device=0,
    project=f"{MODEL}", # 학습한 내용을 저장할 폴더명
    seed=SEED,
    optimizer="AdamW",
    lr0=1e-3,
    hsv_h= 0.015,  # image HSV-Hue augmentation (fraction)
    hsv_s= 0.7,  # image HSV-Saturation augmentation (fraction)
    hsv_v= 0.4,  # image HSV-Value augmentation (fraction)
    degrees= 0.5,  # image rotation (+/- deg)
    translate= 0.1,  # image translation (+/- fraction)
    scale= 0.2,  # image scale (+/- gain)
    fliplr= 0.5, # image flip left-right (probability)
    mosaic= 0.3,  # image mosaic (probability)
    mixup= 0.1  # image mixup (probability)
    )

100%|██████████| 6.23M/6.23M [00:00<00:00, 20.5MB/s]
Ultralytics YOLOv8.0.151 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data/data.yaml, epochs=300, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=16, project=trained_model, name=None, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchs

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]
Overriding model.yaml nc=80 with nc=34

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]         

In [10]:
# 욜로 모델 학습: 두 번째
model = loaded_model # 학습한 모델 불러 와서 추가 학습
print(type(model.names), len(model.names))

print(model.names)
results = model.train(
    data="/content/data/data.yaml", # yaml 데이터가 있는 경로
    imgsz=640,
    epochs=EPOCH,
    batch=BATCH_SIZE,
    # patience=5,
    workers=16,
    device=0,
    project=f"{MODEL}", # 학습한 내용을 저장할 폴더명
    seed=SEED,
    optimizer="AdamW",
    lr0=1e-3,
    hsv_h= 0.015,  # image HSV-Hue augmentation (fraction)
    hsv_s= 0.7,  # image HSV-Saturation augmentation (fraction)
    hsv_v= 0.4,  # image HSV-Value augmentation (fraction)
    degrees= 0.5,  # image rotation (+/- deg)
    translate= 0.1,  # image translation (+/- fraction)
    scale= 0.2,  # image scale (+/- gain)
    fliplr= 0.5, # image flip left-right (probability)
    mosaic= 0.3,  # image mosaic (probability)
    mixup= 0.1  # image mixup (probability)
    )

Ultralytics YOLOv8.0.151 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)


<class 'dict'> 34
{0: 'chevrolet_malibu_sedan_2012_2016', 1: 'chevrolet_malibu_sedan_2017_2019', 2: 'chevrolet_spark_hatchback_2016_2021', 3: 'chevrolet_trailblazer_suv_2021_', 4: 'chevrolet_trax_suv_2017_2019', 5: 'genesis_g80_sedan_2016_2020', 6: 'genesis_g80_sedan_2021_', 7: 'genesis_gv80_suv_2020_', 8: 'hyundai_avante_sedan_2011_2015', 9: 'hyundai_avante_sedan_2020_', 10: 'hyundai_grandeur_sedan_2011_2016', 11: 'hyundai_grandstarex_van_2018_2020', 12: 'hyundai_ioniq_hatchback_2016_2019', 13: 'hyundai_sonata_sedan_2004_2009', 14: 'hyundai_sonata_sedan_2010_2014', 15: 'hyundai_sonata_sedan_2019_2020', 16: 'kia_carnival_van_2015_2020', 17: 'kia_carnival_van_2021_', 18: 'kia_k5_sedan_2010_2015', 19: 'kia_k5_sedan_2020_', 20: 'kia_k7_sedan_2016_2020', 21: 'kia_mohave_suv_2020_', 22: 'kia_morning_hatchback_2004_2010', 23: 'kia_morning_hatchback_2011_2016', 24: 'kia_ray_hatchback_2012_2017', 25: 'kia_sorrento_suv_2015_2019', 26: 'kia_sorrento_suv_2020_', 27: 'kia_soul_suv_2014_2018', 28: 

engine/trainer: task=detect, mode=train, model=/content/trained_model/train/weights/best.pt, data=/content/data/data.yaml, epochs=300, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=16, project=trained_model, name=None, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms

In [11]:
# 테스트 이미지를 배치 사이즈로 변환
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [12]:
# 화면에 표시하는 내용 삭제
from IPython.display import clear_output

In [13]:
# 학습한 모델을 가져 와서 테스트에 적용: predict 폴더 아래 labels란 폴더 만들어서 txt 생성
model = YOLO("/content/trained_model/train2/weights/best.pt")
test_image_paths = glob.glob("/content/data/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=640, iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0: # 5개만 출력함
        clear_output(wait=True)

 95%|█████████▌| 81/85 [00:33<00:01,  2.03it/s]Results saved to trained_model/predict
314 labels saved to trained_model/predict/labels
 96%|█████████▋| 82/85 [00:33<00:01,  1.98it/s]Results saved to trained_model/predict
318 labels saved to trained_model/predict/labels
 98%|█████████▊| 83/85 [00:34<00:01,  2.00it/s]Results saved to trained_model/predict
322 labels saved to trained_model/predict/labels
 99%|█████████▉| 84/85 [00:34<00:00,  1.98it/s]Results saved to trained_model/predict
326 labels saved to trained_model/predict/labels
100%|██████████| 85/85 [00:35<00:00,  2.40it/s]


In [14]:
# txt 파일을 yolo용 txt로 변환
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)

    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [15]:
# yolo용 txt에서 제출 파일용 정보 수집하여 제출 파일 생성
infer_txts = glob.glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"./data/test/{base_file_name}.png").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"./results/{MODEL}.csv", index=False)

100%|██████████| 326/326 [00:17<00:00, 19.13it/s]


In [16]:
import torch

# 학습한 모델 저장
save_path = "/content/drive/MyDrive/Colab Notebooks/21 ~ 24주차 - 3차 프로젝트/yolo_model_5.pth"
torch.save(model, save_path)

In [22]:
shutil.rmtree('/content/results')

In [17]:
# 저장한 모델 불러 오기
loaded_model = torch.load(save_path)
loaded_model

In [18]:
!unzip '/content/drive/MyDrive/Colab Notebooks/21 ~ 24주차 - 3차 프로젝트/test_jpg.zip' -d test

Archive:  /content/drive/MyDrive/Colab Notebooks/21 ~ 24주차 - 3차 프로젝트/test_jpg.zip
   creating: test/content/jpg/test_jpg/
  inflating: test/content/jpg/test_jpg/140513830.jpg  
  inflating: test/content/jpg/test_jpg/093744525.jpg  
  inflating: test/content/jpg/test_jpg/103508707.jpg  
  inflating: test/content/jpg/test_jpg/134208439.jpg  
  inflating: test/content/jpg/test_jpg/065223169.jpg  
  inflating: test/content/jpg/test_jpg/091611858.jpg  
  inflating: test/content/jpg/test_jpg/094045419.jpg  
  inflating: test/content/jpg/test_jpg/132639025.jpg  
  inflating: test/content/jpg/test_jpg/154423106.jpg  
  inflating: test/content/jpg/test_jpg/112353714.jpg  
  inflating: test/content/jpg/test_jpg/155150022.jpg  
  inflating: test/content/jpg/test_jpg/110618501.jpg  
  inflating: test/content/jpg/test_jpg/180209842.jpg  
  inflating: test/content/jpg/test_jpg/114631813.jpg  
  inflating: test/content/jpg/test_jpg/083742000.jpg  
  inflating: test/content/jpg/test_jpg/182732

In [24]:
len(glob.glob('/content/test/content/jpg/test_jpg/*.jpg'))

3400

In [21]:
shutil.rmtree('/content/trained_model/predict')

In [22]:
# 추가 학습한 모델로 테스트 예측
model = loaded_model
test_image_paths = glob.glob("/content/test/content/jpg/test_jpg/*.jpg")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    loaded_model.predict(image, imgsz=640, iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0: # 5개만 출력함
        clear_output(wait=True)

100%|█████████▉| 846/850 [03:20<00:00,  4.38it/s]Results saved to trained_model/predict
3279 labels saved to trained_model/predict/labels
100%|█████████▉| 847/850 [03:20<00:00,  4.40it/s]Results saved to trained_model/predict
3282 labels saved to trained_model/predict/labels
100%|█████████▉| 848/850 [03:20<00:00,  4.51it/s]Results saved to trained_model/predict
3286 labels saved to trained_model/predict/labels
100%|█████████▉| 849/850 [03:21<00:00,  4.52it/s]Results saved to trained_model/predict
3290 labels saved to trained_model/predict/labels
100%|██████████| 850/850 [03:21<00:00,  4.22it/s]


In [29]:
len(test_image_paths)/BATCH_SIZE, len(test_image_paths)

(850.0, 3400)

In [23]:
# yolo용 txt에서 제출 파일용 정보 수집하여 제출 파일 생성
infer_txts = glob.glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"/content/test/content/jpg/test_jpg/{base_file_name}.jpg").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"./results/{MODEL}.csv", index=False)

100%|██████████| 3290/3290 [01:26<00:00, 38.11it/s]
